In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import image as nimg
from nilearn import plotting as nplot
import bids
from nltools.file_reader import onsets_to_dm
from nltools.stats import regress, zscore
from nltools.data import Brain_Data, Design_Matrix
from nltools.stats import find_spikes 
from nilearn.plotting import view_img, glass_brain, plot_stat_map
from bids import BIDSLayout, BIDSValidator
import os
from pathlib import Path
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
from nilearn.plotting import plot_contrast_matrix
from nilearn.glm.first_level import FirstLevelModel
from nilearn import plotting
# creating mean img for plotting purposes 
from nilearn.image import mean_img
from nilearn.image import load_img
from nibabel import load
from nibabel.gifti import GiftiDataArray, GiftiImage
from nilearn.glm.first_level import run_glm as run_glm
from nilearn.glm import compute_contrast
import nilearn
fsaverage = nilearn.datasets.fetch_surf_fsaverage(mesh='fsaverage')

# Making results folder -- change to what this should actually be 
path = '/Volumes/Seagate Desktop Drive/kdata/'
os.chdir(path)
output_dir = Path.cwd() / "results" / "surface" / "first_level_results"
output_dir.mkdir(exist_ok=True, parents=True)
print(f"Output will be saved to: {output_dir}")

#isolating directory called layout_raw (for events.tsv files) -- will change depending on where data is stored / what computer we're running on
layout_raw = bids.BIDSLayout('/Volumes/Seagate Desktop Drive/kdata/', validate=True)

#isolating directory called layout (for fMRIprep derivatives) -- will change depending on where data is stored / what computer we're running on 
layout = bids.BIDSLayout('/Volumes/Seagate Desktop Drive/kdata/derivatives', validate=False,
                  config=['bids','derivatives'])
print(layout)

## Subject-wise parameter estimation for self- other- case- and fix 
creates self-other, self-fix, other-fix, case-fix, self-case contrasts for all subs 

In [ ]:
# needed for making design matrix 
# note: keep MNI files for isolating tr / frames bc easier to load 
def load_bids_events(layout,layout_raw, subject, run, session):
    '''Create a design_matrix instance from BIDS event file'''
    
    tr = 2.5 #put in manually bc get_tr wouldn't work?? 
    # change lines below -- can change to "mask", change task to "self-other"
    func_files = layout.get(subject=subject,
                        datatype='func', task='selfother',session = session,
                        desc='preproc',
                        space='MNI152NLin2009cAsym',
                        extension='nii.gz',
                       return_type='file')
    func_file = nimg.load_img(func_files[run])
    n_tr = func_file.shape[-1]

    onsets = pd.read_csv(layout_raw.get(subject=subject, suffix='events', session = session)[run].path)
    # line below is isolating the onset, duration, and trial type columns -- change according to events.tsv format 
    onsets_actual = onsets.iloc[:, [0,1,3]]
    onsets_actual.columns = ['onset', 'duration','trial_type'] # make sure this order matches with what's loaded in as "onsets_actua
    sampling_freq = 1/tr
    n_scans=n_tr
    return onsets_actual, tr, n_scans

from nilearn.datasets import fetch_icbm152_brain_gm_mask
from nilearn import plotting, masking
from nilearn.masking import _unmask_3d
from nilearn.maskers import nifti_spheres_masker
import nibabel as nib
from nibabel import Nifti1Image
from nilearn.image import resample_to_img 

sess = '01'
sub = '102'
fmri_imgs = layout.get(subject=sub,
            datatype='func', task='selfother',session = sess,
            desc='preproc',
            space='MNI152NLin2009cAsym',
            extension='nii.gz',
           return_type='file')

space_defining_image = masking.compute_brain_mask(fmri_imgs[0])

icbm_mask = fetch_icbm152_brain_gm_mask()
space_defining_image = masking.compute_brain_mask(fmri_imgs[0])

icbm_mask = resample_to_img(source_img=icbm_mask, target_img=space_defining_image, interpolation='nearest')

In [ ]:
subjects = layout.get_subjects()

# subjects missing ses02:
# 102, 138, 145, 209, 221, 237 
subjects2 = []
subjects2 = subjects
#subjects2 = subjects
#testing second level (removing all subs with errors) 
del(subjects2[0]) #- 102 
del(subjects2[21]) #- 138 +1
del(subjects2[25]) #- 145 +1 
del(subjects2[32]) #- 209 +1 
del(subjects2[40]) #- 221 +1 
del(subjects2[50]) #- 237 +2 

subjects = layout.get_subjects()


In [ ]:
# to iterate through subjects ... 
#for sub in subjects 
from scipy.stats import norm

p001_unc = norm.isf(0.001)

file_lists = {"self-other": list(),
    "other-fix": list(),
     "self-fix": list(),
     "case-fix": list(),
     "self-case": list()
    }

firstlevel_plots = {"self-other": list(),
    "other-fix": list(),
     "self-fix": list(),
     "case-fix": list(),
     "self-case": list()                    
    }

nosessions = ['01','02']

for sess in nosessions: 

    if sess == '01':
        subject_list = subjects
    elif sess == '02':
        subject_list = subjects2
    
    for sub in subject_list: 
# change lines below -- can change to "mask", change task to "self-other" -- should match the same format as in the load_bids_events function
        fmri_imgs = layout.get(subject=sub,
                    datatype='func', task='selfother',session = sess,
                    desc='preproc',
                    space='MNI152NLin2009cAsym',
                    extension='nii.gz',
                   return_type='file')
        hrf_model = "spm" #canonical hrf 
        high_pass = 0.01 # The cutoff for the drift model is 0.01 Hz.

        confound_files = layout.get(subject=sub,
                        datatype='func', task='selfother',session = sess,
                        desc='confounds',
                       extension="tsv",
                       return_type='file')

# Select confounds -- set right now to just the 6 motion parameters but can add in more/less! 
       #32 confound_vars -- REMOVING GLOBAL SIGNAL 
        confound_vars = ['trans_x','trans_x_derivative1','trans_x_derivative1_power2','trans_x_power2',
                           'trans_y','trans_y_derivative1','trans_y_derivative1_power2','trans_y_power2',
                             'trans_z','trans_z_derivative1','trans_z_derivative1_power2','trans_z_power2',
                             'rot_x','rot_x_derivative1','rot_x_derivative1_power2','rot_x_power2',
                             'rot_y','rot_y_derivative1','rot_y_derivative1_power2','rot_y_power2',
                             'rot_z','rot_z_derivative1','rot_z_derivative1_power2','rot_z_power2',
                             'csf','csf_derivative1','csf_derivative1_power2','csf_power2',
                             'white_matter','white_matter_derivative1','white_matter_derivative1_power2','white_matter_power2'
                            ]

        final_confounds = confound_vars

        design_matrices = []

        print("Creating First Level Design matrix ... ")

        for idx, img in enumerate(fmri_imgs):
# Build experimental paradigm
            run = idx
            events,tr,n_scans = load_bids_events(layout,layout_raw, sub, run, sess)
    # Define the sampling times for the design matrix
            frame_times = np.arange(n_scans) * tr
            confound_file = confound_files[run]
            confound_df = pd.read_csv(confound_file, delimiter='\t')
            confound_df = confound_df[final_confounds]
            confound_df.fillna(0, inplace=True)
    # Build design matrix with the previously defined parameters
            design_matrix = make_first_level_design_matrix(
                frame_times,
                events,
                hrf_model=hrf_model,
                drift_model="polynomial",
                drift_order=3,
                add_regs=confound_df,
                add_reg_names=confound_vars,
                high_pass=high_pass,
            )
            design_matrix = design_matrix.iloc[:,0:11] # taking out constant intercept and adding in an intercept for each individual run
# this allows average voxel to vary across runs instead of assuming its constant 
            if idx == 0:
                design_matrix['intercept1'] = 1 
                design_matrix['intercept2'] = 0
                design_matrix['intercept3'] = 0
            elif idx == 1:
                design_matrix['intercept1'] = 0
                design_matrix['intercept2'] = 1 
                design_matrix['intercept3'] = 0
            else: 
                design_matrix['intercept1'] = 0
                design_matrix['intercept2'] = 0 
                design_matrix['intercept3'] = 1
    # put the design matrices in a list
            design_matrices.append(design_matrix)
    
# can visualize the design matrix with the line below 
#plot_design_matrix(design_matrices[1])
        print("First Level Design Matrix completed")

        contrast_matrix = np.eye(design_matrix.shape[1])
        basic_contrasts = {
            column: contrast_matrix[i]
            for i, column in enumerate(design_matrix.columns)
        }

        contrasts = {
            "self-other": (basic_contrasts["self"] - basic_contrasts["other"]),
            "self-fix": (basic_contrasts["self"] - basic_contrasts["fix"]),
            "other-fix": (basic_contrasts["other"] - basic_contrasts["fix"]),
            "case-fix": (basic_contrasts["case"] - basic_contrasts["fix"]),
            "self-case": (basic_contrasts["self"] - basic_contrasts["case"])
        }

        print("Fitting first-level GLM ...")
        #added gm mask 
        fmri_glm = FirstLevelModel(mask_img = icbm_mask)
        #no gm mask 
        #fmri_glm = FirstLevelModel()
        
        fmri_glm = fmri_glm.fit(fmri_imgs, design_matrices=design_matrices)

        print("First-level completed!")

        mean_image = mean_img(fmri_imgs[0])

        print("Subject = " + sub)

    # Iterate on contrasts
        for contrast_id, contrast_val in contrasts.items():
    #print(f"\tcontrast id: {contrast_id}")
    # compute the contrasts
            outputs = fmri_glm.compute_contrast(contrast_val, output_type='all')
    # plot the contrasts as soon as they're generated
    # the display is overlaid on the mean fMRI image
    # a threshold of 3.0 is used, more sophisticated choices are possible
            fname = "/Volumes/Seagate Desktop Drive/kdata/results/first_level_results/sub" + sub + "_" + contrast_id + "_ses" + sess + ".nii.gz"
            # saving z-scores for plotting purposes  
            zname = "/Volumes/Seagate Desktop Drive/kdata/results/first_level_results/plotting/sub" + sub + "_" + contrast_id + "_ses" + sess + ".nii.gz"
            nib.save(outputs['effect_size'], fname)
            nib.save(outputs['z_score'], zname)
            plotting.plot_glass_brain(
                outputs['z_score'],
                threshold=p001_unc,
                display_mode="z",
                title=contrast_id,
            )
            plotting.show()
            file_lists[contrast_id].append(fname)
            firstlevel_plots[contrast_id].append(zname)

## Neural Correlates of Self-focused Attention 
self-other contrast at pre-treatment baseline

### Self-other @ pre-treatment: all subjects

In [ ]:
# compiling all of session1 self-other
subject_sessions = {}

# Populate the dictionary
for file_path in file_lists['self-other']:
    base_name = os.path.basename(file_path)
    parts = base_name.split('_')
    subject = parts[0]
    session = parts[-1].split('.')[0]
    if subject not in subject_sessions:
        subject_sessions[subject] = {}
    subject_sessions[subject][session] = file_path

# # isolate just self-fix session1 
self_v_other_pre = [] 
selfvother_pre_results = []
# # Loop through each subject and perform the extraction
for subject, sessions in subject_sessions.items():
    if 'ses01' in sessions:
#         # Load the NIfTI files
        ses01_img = nib.load(sessions['ses01'])
        
        # Append the result to the list
        selfvother_pre_results.append((ses01_img))
        output_file = f'{subject}_selfvother_pre.nii.gz'
        nib.save(ses01_img, output_file)
        self_v_other_pre.append([output_file])
        print(f'Appended result for {subject}')
    else:
        print(f'Subject {subject} does not have ses01 files')

In [ ]:
# second level design matrix 
selfvother_sldm = np.ones(len(self_v_other_pre))
selfvother_sldm = pd.DataFrame(selfvother_sldm, columns=['selfvother'])

In [ ]:
from nilearn.glm.second_level import SecondLevelModel 
second_level_input = selfvother_pre_results
second_level_model = SecondLevelModel()
# plotting.plot_design_matrix(selfvfix_sldm, rescale=False)
# plt.title("Second-level Design Matrix", fontsize=20)
# plotting.show()

from nilearn.glm.second_level import SecondLevelModel

second_level_model = SecondLevelModel(n_jobs=2).fit(second_level_input, design_matrix=selfvother_sldm)
contrast_val = [1]
outputs = second_level_model.compute_contrast(contrast_val, output_type='all')
bkg_img = outputs['stat']
stat_img = outputs['stat']

In [ ]:
# whole brain permutation testing for self-other clusters
from nilearn.glm.second_level import non_parametric_inference

out_dict_selfvother_pre = non_parametric_inference(
    second_level_input,
    design_matrix=selfvother_sldm,
    second_level_contrast=contrast_val,
    n_perm=10000,  # 500 for the sake of time. Ideally, this should be 10,000.
    two_sided_test=True,
    smoothing_fwhm=8.0,
    tfce=True,
    # mask=selfvfix_mask,
    threshold=0.001,
)

In [ ]:
# Load your TFCE result NIfTI file (replace 'result_file.nii.gz' with your actual file path)
result_img = out_dict_selfvother_pre['tfce'] 
# Plotting the TFCE map
plotting.plot_glass_brain(result_img, threshold=0.001, title='TFCE Results', display_mode='ortho',plot_abs=False,
                       colorbar=True)

plt.show()

logp_max_tfce_img = out_dict_selfvother_pre['logp_max_tfce']

threshold = -np.log10(0.001)  # p<0.001 corrected

# Plotting the logp_max_tfce map
plotting.plot_glass_brain(logp_max_tfce_img, threshold=threshold, title='Log P Max TFCE', display_mode='ortho',plot_abs=False,
                        colorbar=True)

# Display the plot
plt.show()

In [ ]:
# cluster summary table / saving thresholded nifti file 

from nilearn.reporting import get_clusters_table
threshold = -np.log10(0.001)  # p<0.001 corrected

logp_max_tfce_img = out_dict_selfvother_pre['logp_max_tfce']

table = get_clusters_table(logp_max_tfce_img, threshold, 30)
print(table.to_latex())

thresh_tfce_img = out_dict_selfvother_pre['logp_max_tfce'].get_fdata()
thresh_tfce_img = (thresh_tfce_img >= 3)

thresh_tfce_img = Nifti1Image(thresh_tfce_img.astype(int), out_dict_selfvother_pre['logp_max_tfce'].affine, header=out_dict_selfvother_pre['logp_max_tfce'].header)

fname = "/Volumes/Seagate Desktop Drive/kdata/results/ms images/selfvsother_allsubs_binarized.nii.gz"
nib.save(thresh_tfce_img, fname)

In [ ]:
# signed, threshold tfce image 

thresh_tfce_img = out_dict_selfvother_pre['logp_max_tfce'].get_fdata()
threshold_mask = (thresh_tfce_img >= 3)

# Extracting t_img data
t_img = out_dict_selfvother_pre['tfce'].get_fdata()

# Applying the threshold mask to t_img
filtered_t_img = np.where(threshold_mask, t_img, 0)  # Use 0 for values that do not surpass the threshold

# # Creating a new NIfTI image with the filtered data
filtered_t_img_nifti = Nifti1Image(filtered_t_img, 
                                   out_dict_selfvother_patientsvcontrols_pre['t'].affine, 
                                   header=out_dict_selfvother_patientsvcontrols_pre['t'].header)
# # Plotting the logp_max_tfce map
plotting.plot_glass_brain(filtered_t_img_nifti, threshold=0, title='Log P Max TFCE', display_mode='ortho',plot_abs=False,
                        colorbar=True)

fname = "/Volumes/Seagate Desktop Drive/kdata/results/ms images/selfvsother_allsubs_tfcevals.nii.gz"
nib.save(filtered_t_img_nifti, fname)